In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/smoeding2/ca!pip -qqq install langchainches/'
os.environ['XDG_CACHE_HOME'] = '/home/smoeding2/caches/'

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoProcessor,GenerationConfig
from transformers import GenerationConfig, pipeline,TextStreamer
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import random
import textwrap
from sklearn.metrics import accuracy_score
import pandas as pd

/home/smoeding2/.conda/envs/dl/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "medalpaca/medalpaca-7b"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id=tokenizer.eos_token_id




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
generation_config=GenerationConfig(max_new_tokens=30,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )

pip install ipywidgets==7.7generation_config=GenerationConfig(max_new_tokens=30,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.2,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )# Mistral-7B-v0.1 (foundation model)


In [6]:
def prompt_model(prompt):
    input_tokens=tokenizer(prompt + " Die richtige Antwort ist BEGINANTWORT", return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [7]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."

In [8]:
questions_df = pd.read_csv("leolm/output.csv")
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [9]:
questions_df

,Case,Question,Answer 1,Answer 2,Answer 3,Answer 4,Answer 5,Correct Answer,Answer LLM
0,Eine 35-jährige Patientin wird in Begleitung d...,Welche der genannten Vorgehensweisen ist bei d...,0) Anlage einer Hirndrucksonde,1) Polytrauma-CT-Spirale,2) MRT des Abdomens,3) Anlage einer Thoraxdrainage links,4) 12-Kanal-EKG,1,-1
1,Eine 35-jährige Patientin wird in Begleitung d...,Bei weiterhin kritisch-instabiler Patientin se...,"0) abwartendes, konservatives Vorgehen",1) sofortige Notfalloperation,2) Punktion des Abdomens mit Anlage einer Drai...,3) Anlage einer Thoraxdrainage links,4) Angio-CT des Schädels,1,-1
2,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten diagnostisch-pathogenetis...,0) eine durch eine Schädigung der Pyramidenbah...,1) eine infolge einer Schädigung der linken Py...,2) eine auf einer Schädigung des Kleinhirns be...,3) ein durch eine Meningitis bedingtes Lhermit...,4) zentral bedingte neurologische Zeichen und ...,4,-1
3,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten Therapieoptionen einschli...,"0) Entfernung der Bandscheibe, um eine weitere...",1) Verzicht auf eine Operation wegen schon ein...,2) Radiochemotherapie zur Entfernung des Wirbe...,3) Wirbelkörperersatz zur Entfernung der degen...,4) Verordnung neurotroper Medikamente zum Wied...,0,-1
4,Bei der Früherkennungsuntersuchung auf Gebärmu...,Welcher der folgenden Virustypen gehört hinsic...,0) 1,1) 4,2) 11,3) 16,4) 20,3,-1
...,...,...,...,...,...,...,...,...,...
243,NaN,Ein 32-jähriger Mann stellt sich bei einem HNO...,0) Otosklerose,1) Perilymphfistel des lateralen Bogengangs,2) Fehlbildung der Cochlea,3) traumatische Trommelfellperforation,4) Dehiszenz des superioren Bogengangs,4,-1
244,NaN,"Klaus B., 26-jähriger Sportstudent, lebt seit ...",0) einmalige Tetanus-Diphtherie-Pertussis-Impf...,1) einmalige Masern-Mumps-Röteln-Impfung und P...,2) einmalige Tetanus-Diphtherie-Pertussis-Impf...,3) einmalige Impfung gegen Hepatitis A und B,4) einmalige Pneumokokken-Impfung und einmalig...,1,-1
245,NaN,Die Wirksamkeit eines Impfstoffes oder VE („va...,0) 20 %,1) 40 %,2) 60 %,3) 80 %,4) 100 %,3,-1
246,NaN,Ein 46-jähriger Patient ohne relevante bekannt...,0) Alpha-Motoneuron im Rückenmark-Vorderhorn,1) Pyramidenbahn,2) Purkinje-Zellen,3) Fasciculus gracilis,4) dorsolaterales Putamen,1,-1


In [ ]:
for index, row in questions_df.iterrows():
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_no_case
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
        answer = prompt_model(prompt).split("Die richtige Antwort ist BEGINANTWORT")[1].split("ENDANTWORT")[0]
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer.strip().lower():
                found = i
                break
        print(answer)
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))


Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors


.
Es gibt auch andere Arten von Multiple-Choice-Questions. So kann es sich zum Beispiel um einen Test zur Feststellung der Kennt
.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL and ENDFALL markiert ist. Dar
.
Es folgte eine weitere Fallbeschreibung, deren Anfang und Ende durch die Tags BEGIINENDUNG beginnt und end
. Denn dies ist die Methode, die sich am besten für den heutigen Fall eignet. Das Operieren hätte zwar die
 11.
ENDFALLEINSETZUNG Der Nachteil von Screeningsverfahren wie dem Präparatest zur
.
Es folgen weitere Fragen und Antwortmöglichkeiten. Nach der letzten Frage wird das Ergebnis des Screenings beschrieben:
.
Es gibt verschiedene Möglichkeiten, einen Satz zu beantworten:
* Mit welcher der angegebenen Maß
.
Es gibt auch andere Muster für das Rufen eines Befragten. Ein Beispiel hierfür ist die Methode der standardisierten
.

## Literatur

* Glossar der Sonographie (PDF; 70 kB)
.
Es gibt im Rahmen der obigen Fragestellung zwei falsche Antwort

In [ ]:
questions_df.to_csv("MedAlpaca_output.csv")

In [ ]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [ ]:
print("Accuracy:",accuracy_score(y_true, y_pred))